## Importing all the libraries

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding, LSTM, Bidirectional,Flatten,Dropout
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

## Fetching and preprocessing the data

In [7]:
def remove_special_chars(rev):
     for remove in map(lambda r: re.compile(re.escape(r)), [",", ":", "\"", "=", "&", ";", "%", "$",
 "@", "^", "*", "(", ")", "{", "}",
 "[", "]", "|", "/", "\\", ">", "<", "-",
 "!", "?", ".", "'",
 "_", "— -", "#"]):
         rev.replace(remove, " ", inplace=True)
     return rev
def remove_tags(text):
     return re.compile(r'<[^>]+>').sub('', text)
def remove_num(text):
     return ''.join(re.sub(r'([0–9]+)','',text))
data = pd.read_csv('/content/drive/MyDrive/IMDB/IMDB_Dataset.csv')
data.review=data.review.apply(lambda x : remove_tags(x))
data.review=data.review.apply(lambda x : remove_num(x))
remove_special_chars(data.review)
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production The filming tec...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there s a family where a little boy ...,negative
4,Petter Mattei s Love in the Time of Money is...,positive


## Word Embeddings

In [8]:
tokenizer = Tokenizer(num_words=5000,filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',lower=True,split=' ')
tokenizer.fit_on_texts(data['review'])
X = tokenizer.texts_to_sequences(data['review'])
X = pad_sequences(X,maxlen=500)
Y = data['sentiment']
vocab_size = len(tokenizer.word_index) + 1


# Creating train and test sets:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state = 24)

#We store this tokenizer in a file to use later in web app
import pickle

# saving
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
from sklearn.preprocessing import LabelEncoder

def prepare_targets(y_train, y_test):
 le = LabelEncoder()
 le.fit(y_train)
 y_train_enc = le.transform(y_train)
 y_test_enc = le.transform(y_test)
 return y_train_enc, y_test_enc
ytrain,ytest = prepare_targets(Y_train,Y_test)

## Designing the Model

In [9]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=500))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',
 loss='binary_crossentropy',
 metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 50)           5275650   
                                                                 
 bidirectional (Bidirectiona  (None, 256)              183296    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 1)                 257       
                                                                 
Total params: 5,459,203
Trainable params: 5,459,203
Non-trainable params: 0
_________________________________________________________________


## Training the model



In [11]:
from keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
history=model.fit(X_train, ytrain,
 batch_size=128,
 epochs=20,
 validation_data=[X_test, ytest],
 callbacks=[es])
#I saved this model to use while deploying
model.save('movie_sent.h5')

Epoch 1/20
274/274 [==============================] - 31s 80ms/step - loss: 0.4847 - accuracy: 0.7604 - val_loss: 0.3377 - val_accuracy: 0.8549
Epoch 2/20
274/274 [==============================] - 20s 75ms/step - loss: 0.2890 - accuracy: 0.8854 - val_loss: 0.3206 - val_accuracy: 0.8613
Epoch 3/20
274/274 [==============================] - 22s 81ms/step - loss: 0.2357 - accuracy: 0.9083 - val_loss: 0.2766 - val_accuracy: 0.8896
Epoch 4/20
274/274 [==============================] - 20s 74ms/step - loss: 0.2171 - accuracy: 0.9171 - val_loss: 0.3055 - val_accuracy: 0.8816
Epoch 5/20
274/274 [==============================] - 20s 74ms/step - loss: 0.1969 - accuracy: 0.9247 - val_loss: 0.2943 - val_accuracy: 0.8837
Epoch 6/20
274/274 [==============================] - 20s 74ms/step - loss: 0.1793 - accuracy: 0.9341 - val_loss: 0.3287 - val_accuracy: 0.8837
Epoch 7/20
274/274 [==============================] - 20s 74ms/step - loss: 0.1570 - accuracy: 0.9412 - val_loss: 0.3310 - val_accuracy:

In [13]:
string11="Between the Lovecraftian overtones and Liberato’s performance, The Beach House offers up beautifully shot terror and will make you think before opening your door."
x_1=tokenizer.texts_to_sequences([string11])
x_1 = pad_sequences(x_1,maxlen=500)
model.predict(x_1)

array([[0.983709]], dtype=float32)

- If the output is closer to 1, then it is a positive review.
- If the output is closer to 0, then it is a negative review.

## Sentiment analysis app using streamlit code

In [22]:
pip install streamlit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.1 MB 28.1 MB/s 
     |████████████████████████████████| 164 kB 67.6 MB/s 
     |████████████████████████████████| 4.7 MB 52.6 MB/s 
     |████████████████████████████████| 78 kB 7.2 MB/s 
     |████████████████████████████████| 181 kB 69.8 MB/s 
     |████████████████████████████████| 235 kB 74.1 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 51 kB 7.8 MB/s 
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19582 sha256=187c223edfbde59ccfe3a09773be5f530268029bc43953757b9c9d181294a9ea
  Stored in directory: /root/.cache/pip/wheels/5f/55/ab/36a76989f7f88d9ca7b1f68da6d94252bb6a8d6ad4f18e04e9
Successfully built validators


In [23]:
import streamlit as st
import pickle
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

def predict(message):

 model=load_model('movie_sent.h5')

 with open('tokenizer.pickle', 'rb') as handle:
  tokenizer = pickle.load(handle)
 x_1 = tokenizer.texts_to_sequences([message])
 x_1 = pad_sequences(x_1, maxlen=500)
 predictions = model.predict(x_1)[0][0]
 return predictions

st.title('Movie Review Sentiment Analyzer')
message = st.text_area('Enter Review:')
if st.button('Analyze'):
 with st.spinner('Analyzing the text …'):
  prediction=predict(message)
 if prediction > 0.6:
  st.success('Positive review with {:.2f} confidence'.format(prediction))
  st.balloons()
 elif prediction <0.4:
  st.error('Negative review with {:.2f} confidence'.format(1-prediction))
 else:
  st.warning('Not sure! Try to add some more words') 

  command:

    streamlit run /usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py [ARGUMENTS]
2022-09-08 06:32:10.093 
  command:

    streamlit run /usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py [ARGUMENTS]
